<a href="https://colab.research.google.com/github/parimalakettymuthu/MachineLearning-Projects/blob/main/MultiLabel_Classification_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys 
sys.path.append('/content/drive/MyDrive/NLP/custom-functions')

In [ ]:
import pandas as pd
from pathlib import Path
import joblib

In [ ]:
if 'google.colab' in str(get_ipython()):
  from google.colab import drive
  drive.mount('/content/drive')   
  base_path = '/content/drive/MyDrive/NLP/assignment6'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data_folder = Path(base_path)

#Check contents in data folder
for entries in data_folder.iterdir():
  print(entries.name)

df_multilabel_hw_cleaned.joblib


In [ ]:
df_multilable_data = data_folder/'df_multilabel_hw_cleaned.pkl'
multiLabel_Data = joblib.load(df_multilable_data)

FileNotFoundError: ignored